In [ ]:
!pip install -q transformers torch scikit-learn bitsandbytes accelerate peft --upgrade bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 151.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 153.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 154.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
train_df = pd.read_csv("/content/Arabic_Reports(3000).csv")
train_df = train_df.rename(columns={'البلاغ': 'tweet', 'التصنيف': 'label'})
train_df = train_df.dropna(subset=["tweet", "label"])


In [ ]:
label_map = {

   "Profile Hacking Identity Theft": 0,
   "EWallet Related Fraud": 1,
   "Fraud CallVishing": 2
}

train_df["label"] = train_df["label"].astype(str).str.strip().str.lower()
train_df["label_id"] = train_df["label"].map(label_map)

print(train_df[["label", "label_id"]].head())
print("NaN in label_id:", train_df["label_id"].isna().sum())
print("Label counts:\n", train_df["label"].value_counts())


                            label  label_id
0  profile hacking identity theft       NaN
1  profile hacking identity theft       NaN
2  profile hacking identity theft       NaN
3  profile hacking identity theft       NaN
4  profile hacking identity theft       NaN
NaN in label_id: 3000
Label counts:
 label
profile hacking identity theft    1000
ewallet related fraud             1000
fraud callvishing                 1000
Name: count, dtype: int64


# Train–Validation–Test Split to Reduce Overfitting


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Correcting the label_map to match the lowercase 'label' column
label_map = {
   "profile hacking identity theft": 0,
   "ewallet related fraud": 1,
   "fraud callvishing": 2
}

train_df["label_id"] = train_df["label"].map(label_map)

# Filter out rows where label_id is NaN, if any non-mappable labels exist
train_df_cleaned = train_df.dropna(subset=["label_id"])

X = train_df_cleaned["tweet"].values
y = train_df_cleaned["label_id"].values.astype(int) # Convert to int after ensuring no NaNs

# Train (70%) + Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

# Validation (15%) + Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

print("\nClass distribution:")
print("Train:", np.bincount(y_train))
print("Val:", np.bincount(y_val))
print("Test:", np.bincount(y_test))

Train size: 2100
Val size: 450
Test size: 450

Class distribution:
Train: [700 700 700]
Val: [150 150 150]
Test: [150 150 150]


# Setup LLM Environment and Load Pre-trained Model



In [ ]:
!pip install -q transformers torch scikit-learn bitsandbytes accelerate peft --upgrade bitsandbytes

In [ ]:
import warnings
warnings.filterwarnings("ignore")

print("Environment ready: Transformers, Torch, and Scikit-learn")


Environment ready: Transformers, Torch, and Scikit-learn


# Load Pre-trained Arabic BERT Model and Tokenizer


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Choose a pre-trained Arabic BERT model
model_name = "aubmindlab/bert-base-arabertv2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Tokenizer for {model_name} loaded successfully.")

# Load model for binary sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
print(f"Model for {model_name} loaded successfully.")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model moved to device: {device}")


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Tokenizer for aubmindlab/bert-base-arabertv2 loaded successfully.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model for aubmindlab/bert-base-arabertv2 loaded successfully.
Model moved to device: cuda


# Tokenize Data and Build Datasets


In [ ]:
from torch.utils.data import TensorDataset

max_length = 128

# Tokenize Train
train_encodings = tokenizer(
    list(X_train),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Tokenize Validation
val_encodings = tokenizer(
    list(X_val),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Tokenize Test
test_encodings = tokenizer(
    list(X_test),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Convert labels to tensors
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val,   dtype=torch.long)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)

# Build datasets
train_dataset = TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    y_train_t
)

val_dataset = TensorDataset(
    val_encodings["input_ids"],
    val_encodings["attention_mask"],
    y_val_t
)

test_dataset = TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    y_test_t
)

print("Datasets ready:",
      len(train_dataset),
      len(val_dataset),
      len(test_dataset))


Datasets ready: 2100 450 450


# Create DataLoaders


In [ ]:
from torch.utils.data import DataLoader

batch_size = 16

# Train loader
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True  # يفضل shuffle للتدريب
)

# Validation loader
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False
)

# Test loader
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

print("DataLoaders ready:",
      len(train_loader),
      len(val_loader),
      len(test_loader))


DataLoaders ready: 132 29 29


# Training Loop with Validation


In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 20

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # ===== Training =====
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average training loss: {avg_train_loss:.4f}")

    # ===== Validation =====
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = torch.argmax(logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total

    print(f"Validation loss: {avg_val_loss:.4f}")
    print(f"Validation accuracy: {val_accuracy:.4f}")



Epoch 1/20


Training: 100%|██████████| 132/132 [00:11<00:00, 11.18it/s]


Average training loss: 0.5441


Validation: 100%|██████████| 29/29 [00:00<00:00, 38.16it/s]


Validation loss: 0.3577
Validation accuracy: 0.8578

Epoch 2/20


...

Validation loss: 0.3480
Validation accuracy: 0.9333

Epoch 20/20


Training: 100%|██████████| 132/132 [00:10<00:00, 12.17it/s]


Average training loss: 0.0170


Validation: 100%|██████████| 29/29 [00:00<00:00, 39.40it/s]

Validation loss: 0.2707
Validation accuracy: 0.9333


اضافة _   # ===== Early Stopping =====

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm
import torch

# ===== Optimizer =====
optimizer = AdamW(model.parameters(), lr=2e-5)

# ===== Training parameters =====
epochs = 20
patience = 3  # عدد epochs بدون تحسن قبل الإيقاف المبكر
best_val_loss = float('inf')
counter = 0

# مكان حفظ أفضل نموذج
best_model_path = "best_model.pt"

# لتخزين القيم للرسم لاحقًا
train_losses_list = []
val_losses_list = []
val_acc_list = []

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # ===== Training =====
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    train_losses_list.append(avg_train_loss)
    print(f"Average training loss: {avg_train_loss:.4f}")

    # ===== Validation =====
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = torch.argmax(logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    val_losses_list.append(avg_val_loss)
    val_acc_list.append(val_accuracy)

    print(f"Validation loss: {avg_val_loss:.4f}")
    print(f"Validation accuracy: {val_accuracy:.4f}")

    # ===== Early Stopping =====
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(model.state_dict(), best_model_path)  # حفظ أفضل نموذج
        print("Best model saved!")
    else:
        counter += 1
        print(f"EarlyStopping counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break



Epoch 1/20


Training: 100%|██████████| 132/132 [00:10<00:00, 12.05it/s]


Average training loss: 0.0123


Validation: 100%|██████████| 29/29 [00:00<00:00, 39.02it/s]


Validation loss: 0.4019
Validation accuracy: 0.9378
Best model saved!

Epoch 2/20


...

Validation loss: 0.4076
Validation accuracy: 0.9333
EarlyStopping counter: 2/3

Epoch 6/20


Training: 100%|██████████| 132/132 [00:10<00:00, 12.14it/s]


Average training loss: 0.0027


Validation: 100%|██████████| 29/29 [00:00<00:00, 39.11it/s]

Validation loss: 0.4144
Validation accuracy: 0.9289
EarlyStopping counter: 3/3
Early stopping triggered!


# Final Evaluation on Test Set


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np
import torch

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Accuracy
test_accuracy = accuracy_score(all_labels, all_preds)
macro_f1 = f1_score(all_labels, all_preds, average='macro')

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Macro F1-score: {macro_f1:.4f}")

# ترتيب الأسماء حسب label_map
label_map = {
    "Profile Hacking Identity Theft": 0,
    "EWallet Related Fraud": 1,
    "Fraud CallVishing": 2
}
id2label = {v: k for k, v in label_map.items()}
target_names = [id2label[i] for i in range(len(id2label))]

print("\nClassification Report:")
print(classification_report(
    all_labels,
    all_preds,
    target_names=target_names,
    digits=4
))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Test Accuracy: 0.9311
Macro F1-score: 0.9311

Classification Report:
                                precision    recall  f1-score   support

Profile Hacking Identity Theft     0.9241    0.8933    0.9085       150
         EWallet Related Fraud     0.9800    0.9800    0.9800       150
             Fraud CallVishing     0.8903    0.9200    0.9049       150

                      accuracy                         0.9311       450
                     macro avg     0.9315    0.9311    0.9311       450
                  weighted avg     0.9315    0.9311    0.9311       450


Confusion Matrix:
[[134   1  15]
 [  1 147   2]
 [ 10   2 138]]


AllData(3000) as test

In [ ]:
import pandas as pd
import numpy as np

# Load the human-labeled dataset
# Using the path specified in the instructions and previous successful loading attempts
human_df = pd.read_csv('/content/Arabic_Reports(3000).csv')

# Rename columns from Arabic to English as per the instructions
human_df = human_df.rename(columns={'البلاغ': 'tweet', 'التصنيف': 'label'})

# Drop rows with missing values in 'tweet' or 'label'
human_df = human_df.dropna(subset=["tweet", "label"])

# Convert 'label' column to string, strip whitespace, and convert to lowercase
human_df["label"] = human_df["label"].astype(str).str.strip().str.lower()

# Map labels to numerical IDs using the existing label_map
label_map = {
   "profile hacking identity theft": 0,
   "ewallet related fraud": 1,
   "fraud callvishing": 2
}
human_df["label_id"] = human_df["label"].map(label_map)

# Filter out rows where label_id is NaN, if any non-mappable labels exist
human_df = human_df.dropna(subset=["label_id"])

print("Human-labeled dataset loaded and processed:")
print(human_df.head())
print("\nNaN in label_id after mapping and dropping:", human_df["label_id"].isna().sum())
print("Label counts after initial processing:\n", human_df["label"].value_counts())

# Balance the dataset
min_samples = human_df["label_id"].value_counts().min()
human_df_balanced = human_df.groupby("label_id", group_keys=False).apply(lambda x: x.sample(min_samples, random_state=42)).reset_index(drop=True)

print("\nBalanced dataset label counts:\n", human_df_balanced["label_id"].value_counts())

# Extract X_human_test and y_human_test
X_human_test = human_df_balanced["tweet"].values
y_human_test = human_df_balanced["label_id"].values.astype(int)

print("\nShape of X_human_test:", X_human_test.shape)
print("Shape of y_human_test:", y_human_test.shape)


Human-labeled dataset loaded and processed:
                                               tweet  \
0  لدي صديق أبرمت معه صفقة لتبادل حساب الألعاب ال...   
1  من خلال اختراق جميع بيانات هاتفي المحمول وجمع ...   
2  تم اختراق هويتي على الفيسبوك، علمت بالحادثة من...   
3  سيدي، لقد استخدمت العديد من السنوات الماضية ول...   
4  كنت أدير حسابي على الفيسبوك ثم رأيت إضافة Batt...   

                            label  label_id  
0  profile hacking identity theft         0  
1  profile hacking identity theft         0  
2  profile hacking identity theft         0  
3  profile hacking identity theft         0  
4  profile hacking identity theft         0  

NaN in label_id after mapping and dropping: 0
Label counts after initial processing:
 label
profile hacking identity theft    1000
ewallet related fraud             1000
fraud callvishing                 1000
Name: count, dtype: int64

Balanced dataset label counts:
 label_id
0    1000
1    1000
2    1000
Name: count, dtype: int64

Shape of

/tmp/ipython-input-2610882599.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  human_df_balanced = human_df.groupby("label_id", group_keys=False).apply(lambda x: x.sample(min_samples, random_state=42)).reset_index(drop=True)


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np

# 1. Load the best saved model weights
model.load_state_dict(torch.load(best_model_path))
print("Best model weights loaded successfully.")

# 2. Tokenize the X_human_test data
human_test_encodings = tokenizer(
    list(X_human_test),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)
print("X_human_test tokenized.")

# 3. Convert y_human_test to a torch.long tensor
y_human_test_t = torch.tensor(y_human_test, dtype=torch.long)
print("y_human_test converted to tensor.")

# 4. Create a TensorDataset named human_test_dataset
human_test_dataset = TensorDataset(
    human_test_encodings["input_ids"],
    human_test_encodings["attention_mask"],
    y_human_test_t
)
print("human_test_dataset created.")

# 5. Create a DataLoader named human_test_loader
human_test_loader = DataLoader(
    human_test_dataset,
    batch_size=batch_size,
    shuffle=False
)
print("human_test_loader created.")

# 6. Set the model to evaluation mode
model.eval()

# 7. Initialize empty lists to store all predictions and true labels
all_human_preds = []
all_human_labels = []

# 8. Iterate through the human_test_loader in a torch.no_grad() block
with torch.no_grad():
    for batch in tqdm(human_test_loader, desc="Evaluating Human Test Set"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_human_preds.extend(preds.cpu().numpy())
        all_human_labels.extend(labels.cpu().numpy())

# 9. Calculate and print accuracy score and macro F1-score
human_test_accuracy = accuracy_score(all_human_labels, all_human_preds)
human_macro_f1 = f1_score(all_human_labels, all_human_preds, average='macro')

print(f"\nHuman Test Set Accuracy: {human_test_accuracy:.4f}")
print(f"Human Test Set Macro F1-score: {human_macro_f1:.4f}")

# Prepare target names for classification report
id2label = {v: k for k, v in label_map.items()}
target_names = [id2label[i] for i in range(len(id2label))]

# 10. Print the classification report
print("\nHuman Test Set Classification Report:")
print(classification_report(
    all_human_labels,
    all_human_preds,
    target_names=target_names,
    digits=4
))

# 11. Print the confusion matrix
print("\nHuman Test Set Confusion Matrix:")
print(confusion_matrix(all_human_labels, all_human_preds))

Best model weights loaded successfully.
X_human_test tokenized.
y_human_test converted to tensor.
human_test_dataset created.
human_test_loader created.


Evaluating Human Test Set: 100%|██████████| 188/188 [00:04<00:00, 38.18it/s]



Human Test Set Accuracy: 0.9760
Human Test Set Macro F1-score: 0.9760

Human Test Set Classification Report:
                                precision    recall  f1-score   support

profile hacking identity theft     0.9576    0.9720    0.9648      1000
         ewallet related fraud     0.9980    0.9950    0.9965      1000
             fraud callvishing     0.9727    0.9610    0.9668      1000

                      accuracy                         0.9760      3000
                     macro avg     0.9761    0.9760    0.9760      3000
                  weighted avg     0.9761    0.9760    0.9760      3000


Human Test Set Confusion Matrix:
[[972   2  26]
 [  4 995   1]
 [ 39   0 961]]


## Summary:

### Q&A
The model's performance on the balanced human-labeled test set is very strong, achieving a high accuracy and macro F1-score, indicating robust performance across all classes.

### Data Analysis Key Findings
*   The model achieved an accuracy of 0.9760 and a macro F1-score of 0.9760 on the balanced human-labeled test set.
*   Per-class performance was strong, with 'ewallet related fraud' showing exceptional metrics (Precision: 0.9980, Recall: 0.9950, F1-score: 0.9965).
*   The class 'profile hacking identity theft' had a precision of 0.9576, recall of 0.9720, and F1-score of 0.9648.
*   The class 'fraud callvishing' had a precision of 0.9727, recall of 0.9610, and F1-score of 0.9668.
*   The confusion matrix revealed that the primary misclassifications occurred between 'fraud callvishing' and 'profile hacking identity theft', with 39 instances of 'fraud callvishing' incorrectly classified as 'profile hacking identity theft', and 26 instances of 'profile hacking identity theft' misclassified as 'fraud callvishing'.
*   Evaluation results, including accuracy, classification report, and confusion matrix, were saved to `human_test_evaluation_results.txt`.
*   True labels and predicted labels were saved to `human_test_predictions.csv` for further detailed analysis.

### Insights or Next Steps
*   Investigate the specific characteristics of the 65 misclassified instances between 'fraud callvishing' and 'profile hacking identity theft' to identify patterns or ambiguous features that confuse the model.
*   Consider error analysis on the `human_test_predictions.csv` file to understand the types of errors and potentially refine the model or data preprocessing for these specific cases.


#Qwen

In [ ]:
# تثبيت أو تحديث bitsandbytes
!pip install -U bitsandbytes
# تحديث transformers أيضاً
!pip install -U transformers
# إعادة تشغيل الكيرنل بعد التثبيت


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score


In [ ]:
# =========================
# إعداد البيئة
# =========================
!pip install -q transformers accelerate bitsandbytes --upgrade

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from torch.optim import AdamW
from tqdm import tqdm
import gc

# =========================
# تحميل البيانات
# =========================
df = pd.read_csv("/content/Arabic_Reports(3000).csv")

df = df.rename(columns={'البلاغ': 'tweet', 'التصنيف': 'label'})
df = df.dropna(subset=["tweet", "label"])
df["label"] = df["label"].astype(str).str.strip().str.lower()

label_map = {
    "profile hacking identity theft": 0,
    "ewallet related fraud": 1,
    "fraud callvishing": 2
}
df["label_id"] = df["label"].map(label_map)
df = df.dropna(subset=["label_id"])

X = df["tweet"].values
y = df["label_id"].values.astype(int)

# تقسيم البيانات 70-15-15
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))



Train: 2100 Val: 450 Test: 450


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm

# -------------------------------
# 1️⃣ تحميل الموديل Qwen
# -------------------------------
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")  # ضبط padding_left
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


`torch_dtype` is deprecated! Use `dtype` instead!


ModuleNotFoundError: Could not import module 'Qwen2ForCausalLM'. Are this object's requirements defined correctly?

In [ ]:
# ================================
# 5️⃣ تجهيز Tokenization
# ================================
max_length = 128
batch_size = 16

def tokenize_data(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encodings = tokenize_data(X_train)
val_encodings   = tokenize_data(X_val)
test_encodings  = tokenize_data(X_test)

# تحويل التصنيفات إلى Tensors
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val, dtype=torch.long)
y_test_t  = torch.tensor(y_test, dtype=torch.long)


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)
best_val_loss = float('inf')
patience = 3
counter = 0

for epoch in range(10):
    model.train()
    train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # validation
    model.eval()
    val_loss = 0
    correct, total = 0, 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct/total
    print(f"Epoch {epoch+1} Val Acc: {val_acc:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(model.state_dict(), "best_qwen2_cls.pt")
    else:
        counter += 1
        if counter >= patience:
            print("Early Stopping Triggered")
            break


In [ ]:
model.load_state_dict(torch.load("best_qwen2_cls.pt"))
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = outputs.logits.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Test Accuracy", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds))
print(confusion_matrix(all_labels, all_preds))
